| Model               | FiveVTS | Chickenpox | Pedalme | Wikimath | Windmillsmall | MontevideoBus |
|---------------------|---------|------------|---------|----------|---------------|---------------|
| Max iter.           | 30      | 30         | 30      | 30       | 30            | 30            |
| Epochs              | 50      | 50         | 50      | 50       | 50            | 50            |
| Lags                | 2       | 4          | 4       | 8        | 8             | 4             |
| Interpolation       | linear  | linear     | nearest | linear   | linear        | nearest       |
| Filters             |         |            |         |          |               |               |
| GConvGRU            | 12      | 16         | 12      | 12       | 12            | 12            |
| GConvLSTM           | 12      | 32         | 2       | 64       | 16            | 12            |
| GCLSTM              | 4       | 16         | 4       | 64       | 16            | 12            |
| LRGCN               | 4       | 8          | 8       | 32       | 12            | 2             |
| DyGrEncoder         | 12      | 12         | 12      | 12       | 12            | 12            |
| EvolveGCNH          | No need | No need    | No need | No need  | No need       | No need       |
| EvolveGCNO          | No need | No need    | No need | No need  | No need       | No need       |
| TGCN                | 12      | 12         | 12      | 12       | 12            | 8             |
| DCRNN               | 2       | 16         | 8       | 12       | 4             | 12            |


In [3]:
import pandas as pd

import itstgcnTGCN
import torch
import itstgcnTGCN.planner 

In [4]:
from torch_geometric_temporal.dataset import PedalMeDatasetLoader
loader2 = PedalMeDatasetLoader()

In [5]:
plans_stgcn_rand = {
    'max_iteration': 30, 
    'method': ['STGCN', 'IT-STGCN'],
    'mrate': [0],
    'lags': [4], 
    'nof_filters': [12], 
    'inter_method': ['linear'],
    'epoch': [50]
}

In [ ]:
plnr = itstgcnTGCN.planner.PLNR_STGCN_RAND(plans_stgcn_rand,loader2,dataset_name='pedalme')
plnr.simulate()

1/30 is done
2/30 is done
3/30 is done
4/30 is done
5/30 is done
6/30 is done
7/30 is done
8/30 is done
9/30 is done
10/30 is done
11/30 is done
12/30 is done
13/30 is done
14/30 is done
15/30 is done
16/30 is done
17/30 is done
18/30 is done
19/30 is done
20/30 is done
21/30 is done
22/30 is done
23/30 is done
24/30 is done
25/30 is done
26/30 is done
27/30 is done
28/30 is done


In [ ]:
plnr = itstgcnTGCN.planner.PLNR_STGCN_RAND(plans_stgcn_rand,loader2,dataset_name='pedalme')
plnr.simulate()

In [ ]:
pd.concat([pd.DataFrame('./simulation)results/2024-02-04_09-21-48.csv'),
          pd.DataFrame('./simulation)results/2024-02-04_09-26-46.csv'),
          pd.DataFrame('./simulation)results/2024-02-04_09-31-36.csv'),
          pd.DataFrame('./simulation)results/.csv'),
          pd.DataFrame('./simulation)results/.csv'),
          ]).groupby(['method']).mean('mse')

# Check

---

In [3]:
import torch_geometric_temporal

In [4]:
dataset = loader2.get_dataset(lags=4)

In [5]:
train_dataset, test_dataset = torch_geometric_temporal.signal.temporal_signal_split(dataset, train_ratio=0.8)

In [6]:
mtype = None
inter_method = None 

In [7]:
method = 'STGCN'
lrnr = itstgcnTGCN.StgcnLearner(train_dataset,dataset_name='pedalme')
lrnr.learn(filters=12,epoch=1)
evtor = itstgcnTGCN.Evaluator(lrnr,train_dataset,test_dataset)
evtor.calculate_mse()
evtor.mse['test']['total']

/home/csy/Dropbox/ITTGNN_blog/posts/2_research/itstgcnTGCN/learners.py:84: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343998658/work/torch/csrc/utils/tensor_new.cpp:245.)
  self.lags = torch.tensor(train_dataset.features).shape[-1]


1.337457299232483

In [8]:
method = 'IT-STGCN'
lrnr = itstgcnTGCN.ITStgcnLearner(train_dataset,dataset_name='pedalme')
lrnr.learn(filters=12,epoch=1)
evtor = itstgcnTGCN.Evaluator(lrnr,train_dataset,test_dataset)
evtor.calculate_mse()
evtor.mse['test']['total']

1.200137734413147

---

In [81]:
import itstgcnTGCN

In [82]:
from torch_geometric_temporal.dataset import PedalMeDatasetLoader
loader2 = PedalMeDatasetLoader()

In [83]:
import torch_geometric_temporal

In [85]:
dataset = loader2.get_dataset(lags=4)

In [86]:
train_dataset, test_dataset = torch_geometric_temporal.signal.temporal_signal_split(dataset, train_ratio=0.8)

In [87]:
f,lags = itstgcnTGCN.convert_train_dataset(train_dataset)

In [94]:

from rpy2.robjects.vectors import FloatVector
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri as rpyn
GNAR = importr('GNAR') # import GNAR 
#igraph = importr('igraph') # import igraph 

ebayesthresh = importr('EbayesThresh').ebayesthresh

In [95]:
f = np.array(f)
if len(f.shape)==1: f = f.reshape(-1,1)
T,N = f.shape
Psi = itstgcnTGCN.make_Psi(T)
fbar = Psi.T @ f # apply dft 
fbar_threshed = np.stack([ebayesthresh(FloatVector(fbar[:,i])) for i in range(N)],axis=1)
fhat = Psi @ fbar_threshed # inverse dft 

In [100]:
itstgcnTGCN.update_from_freq_domain(f,None) - f

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.